In [1]:
import Pkg
Pkg.activate("../../tensor")
import tensor as ten
using ITensors
import LinearAlgebra as LA

  Activating project at `c:\Users\kakin\home\workspace\tensor`


In [2]:
#Index
#大きな足
α = Index(2, "α")
ξ = Index(2, "ξ")
β = Index(2, "β")
η = Index(2, "η")

#小さな足
i = Index(2, "i")
j = Index(2, "j")
k = Index(2, "k")
l = Index(2, "l")

#つぶす足
c1 = Index(2 , "c1")
c2 = Index(2 , "c2")

#定数K
K_const=-0.1

-0.1

In [3]:
#ここがおかしい
#4脚テンソルの定義
W = ten.four_leg_tensor_def(i, j, k, l,K_const)
#2脚テンソルの定義
C = ten.two_leg_tensor_def(α, β, c1, c2, K_const)
#3脚テンソルの定義
PR = ten.three_leg_tensor_def(α, ξ, l, c1, K_const)
#3脚テンソルの定義
PC = ten.three_leg_tensor_def(β, η , k, c1, K_const)

ITensor ord=3 (dim=2|id=181|"β") (dim=2|id=652|"η") (dim=2|id=427|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [4]:
#3脚テンソル(横)の拡大
PR2,ξ,α,ξi=ten.Expand_PR(PR,W,α,k,ξ,i)
#3脚テンソル(縦)の拡大
PC2,η,β,ηj=ten.Expand_PC(PC,W,β,l,η,j)
#角転送行列の拡大
C2=ten.Expand_C(PR,C,PC,W,ξi,ηj,α,β)

ITensor ord=2 (dim=4|id=259|"α") (dim=4|id=240|"β")
NDTensors.Dense{Float64, Vector{Float64}}

In [5]:
function Diagonal_C_matrix(C2::ITensor,α::Index{Int64},β::Index{Int64})
    # 固有値分解
    D, U = eigen(C2, α, β)

    # インデックスの取り出し
    dl = uniqueind(D, U)
    dr = commonind(D, U)

    # 正しい方法でUlを作成（置き換えは個別に！）
    Ul = replaceinds(U, (β => α, dr => dl))

    return D,U,Ul
end

D,U,Ul=Diagonal_C_matrix(C2,α,β)

# 誤差チェック
diff = norm(C2 * U - Ul * D)
println("norm(C2 * U - Ul * D) = ", diff)
println("一致する？ ", diff < 1e-11)



norm(C2 * U - Ul * D) = 1.1791567914286052e-12
一致する？ true


In [6]:
@show D

D = ITensor ord=2
Dim 1: (dim=4|id=611|"Link,eigen")'
Dim 2: (dim=4|id=611|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}
 4×4
 -111.7235422814734 + 0.0im                  0.0 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                0.0 + 0.0im  -2.2167027949896285 + 0.0im                  0.0 + 0.0im                 0.0 + 0.0im
                0.0 + 0.0im                  0.0 + 0.0im  0.22317659368484247 + 0.0im                 0.0 + 0.0im
                0.0 + 0.0im                  0.0 + 0.0im                  0.0 + 0.0im  1109.6947235926991 + 0.0im


ITensor ord=2 (dim=4|id=611|"Link,eigen")' (dim=4|id=611|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}

In [7]:
function Sort_Diagonal(D::ITensor)
    Dnew=copy(D)

    #対角要素の並び替え
    for i in 1:size(D)[1]
        for j in (i+1):size(D)[2]
            if abs(real(Dnew[i,i])) < abs(real(Dnew[j,j]))
                tmp=Dnew[i,i]
                Dnew[i,i]=Dnew[j,j]
                Dnew[j,j]=tmp
            end
        end
    end

    
    return Dnew
end

Sort_Diagonal (generic function with 1 method)

In [8]:
function Sort_idx(D::ITensor,i::Index{Int64},j::Index{Int64})
    # 対角要素を取り出す
    diag_elements=Float64[]
    for i in 1:size(D)[1]
        push!(diag_elements,D[i,i])
    end
    
    # 絶対値でソートしたインデックスを取得
    sorted_indices = sortperm(abs.(diag_elements),rev=true)
    
    return sorted_indices
end

Sort_idx (generic function with 1 method)

In [9]:
χ_num=2
χ=Index(χ_num,"χ")

(dim=2|id=508|"χ")

In [10]:
#固有値をソートし並べ替え、制限した後の対角行列を返す
function Restrict_Diagonal(D::ITensor, χ::Int64,a1::Index{Int64},a2::Index{Int64})
    sortedD=Sort_Diagonal(D)
    tmp1=Index(size(D)[1],"t1")
    tmp2=Index(size(D)[2],"t2")
    idx=Sort_idx(D::ITensor,tmp1::Index{Int64},tmp2::Index{Int64})

    #制限の必要がなければ返す
    if size(D)[1]<χ
        return sortedD ,idx
    end

    resD=ITensor(a1',a2)

    for i in 1:χ
        resD[i,i]=sortedD[i,i]
    end

    return resD,idx
end
resD,idx = Restrict_Diagonal(D, χ_num,χ,χ)
@show resD
@show idx


resD = ITensor ord=2
Dim 1: (dim=2|id=508|"χ")'
Dim 2: (dim=2|id=508|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
 1109.6947235926991 + 0.0im                 0.0 + 0.0im
                0.0 + 0.0im  -111.7235422814734 + 0.0im
idx = [4, 1, 2, 3]


4-element Vector{Int64}:
 4
 1
 2
 3

In [11]:
function Restrict_EigenvecsU(U::ITensor, χ::Int64,a1::Index{Int64},a2::Index{Int64},idx::Vector{Int64})
    #並び替え
    newU=copy(U)
    
    for i in 1:size(U)[1]
        cou=1
        for j in idx
            newU[i,cou]=U[i,j]
            cou+=1
        end
    end
    
    #制限の必要がなければ返す
    if size(U)[1]<χ
        return newU
    end

    #大きさを制限
    resU=ITensor(a1,a2)
    for i in 1:size(U)[1]
        for j in 1:χ
            resU[i,j]=newU[i,j]
        end
    end

    return resU
end
resU = Restrict_EigenvecsU(U, χ_num,β,χ,idx)
@show U
@show resU

U = ITensor ord=2
Dim 1: (dim=4|id=240|"β")
Dim 2: (dim=4|id=611|"Link,eigen")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×4
  0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im  0.49490655590318317 + 0.0im   0.5050420783697566 + 0.0im
 0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im  -0.5050420783697509 + 0.0im    0.494906555903189 + 0.0im
 -0.4896609386040415 + 0.0im  -0.5101295572748379 + 0.0im   -0.505042078369763 + 0.0im  0.49490655590318916 + 0.0im
 -0.5101295572748253 + 0.0im   0.4896609386040544 + 0.0im  0.49490655590319454 + 0.0im    0.505042078369757 + 0.0im
resU = ITensor ord=2
Dim 1: (dim=4|id=240|"β")
Dim 2: (dim=2|id=508|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×2
  0.5050420783697566 + 0.0im   0.5101295572748251 + 0.0im
   0.494906555903189 + 0.0im  0.48966093860404125 + 0.0im
 0.49490655590318916 + 0.0im  -0.4896609386040415 + 0.0im
   0.505042078369757 + 0.0im  -0.5101295572748253 + 0.0im


ITensor ord=2 (dim=4|id=240|"β") (dim=2|id=508|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [12]:
function Restrict_EigenvecsUl(Ul::ITensor, χ::Int64,a1::Index,a2::Index)
    #並び替え
    newUl=copy(Ul)
    
    for i in 1:size(Ul)[1]
        cou=1
        for j in idx
            newUl[i,cou]=Ul[i,j]
            cou+=1
        end
    end

    #制限の必要がなければ返す
    if size(Ul)[1]<χ
        return Ul
    end

    #大きさを制限
    resUl=ITensor(a2',a1)
    for i in 1:χ
        for j in 1:size(Ul)[2]
            resUl[i,j]=newUl[i,j]
        end
    end

    return resUl
end
resUl = Restrict_EigenvecsUl(Ul, χ_num,α,χ)
@show Ul
@show resUl

Ul = ITensor ord=2
Dim 1: (dim=4|id=259|"α")
Dim 2: (dim=4|id=611|"Link,eigen")'
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×4
  0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im  0.49490655590318317 + 0.0im   0.5050420783697566 + 0.0im
 0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im  -0.5050420783697509 + 0.0im    0.494906555903189 + 0.0im
 -0.4896609386040415 + 0.0im  -0.5101295572748379 + 0.0im   -0.505042078369763 + 0.0im  0.49490655590318916 + 0.0im
 -0.5101295572748253 + 0.0im   0.4896609386040544 + 0.0im  0.49490655590319454 + 0.0im    0.505042078369757 + 0.0im
resUl = ITensor ord=2
Dim 1: (dim=2|id=508|"χ")'
Dim 2: (dim=4|id=259|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×4
 0.5050420783697566 + 0.0im   0.5101295572748251 + 0.0im  -0.4896609386040289 + 0.0im  0.49490655590318317 + 0.0im
  0.494906555903189 + 0.0im  0.48966093860404125 + 0.0im   0.5101295572748119 + 0.0im  -0.5050420783697509 + 0.0im


ITensor ord=2 (dim=2|id=508|"χ")' (dim=4|id=259|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [17]:
PR2|>display
resUl|>display

resPR=PR2*resUl

ITensor ord=3 (dim=4|id=266|"ξ") (dim=2|id=722|"j") (dim=4|id=259|"α")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=2 (dim=2|id=508|"χ")' (dim=4|id=259|"α")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

ITensor ord=3 (dim=4|id=266|"ξ") (dim=2|id=722|"j") (dim=2|id=508|"χ")'
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [18]:
PC2|>display
resU|>display

resPC=PC2*resU

ITensor ord=3 (dim=4|id=882|"η") (dim=2|id=401|"i") (dim=4|id=240|"β")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=2 (dim=4|id=240|"β") (dim=2|id=508|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

ITensor ord=3 (dim=4|id=882|"η") (dim=2|id=401|"i") (dim=2|id=508|"χ")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [21]:
resPC*resD*resPR

ITensor ord=4 (dim=4|id=882|"η") (dim=2|id=401|"i") (dim=4|id=266|"ξ") (dim=2|id=722|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [15]:
function Restrict_PR2(PR2::ITensor,α::Index{Int64},)
    
end


Restrict_PR2 (generic function with 1 method)